In [23]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Imports

In [36]:
%matplotlib inline

import IPython.display
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import copy
from scipy.io import wavfile
from scipy.signal import butter, lfilter
import scipy.ndimage
import librosa

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from utils import log_textfile, get_mean_std, windows, pad_dimesions_mfcc, load_model
from basetrainer import BaseTrainer
from dataloader import load_data
from CustomDataset import CustomDatasetSimple
from CustomDatasetMFCC import CustomDatasetMFCC
from models.resnet import ResNet, resnet34
from attacks import fgsm, pgd_linf, fgsm_targ, pgd_linf_targ

In [27]:
DATADIR = '../../Data/'

data = load_data(DATADIR)

In [28]:
# load file names

trainset = pd.read_csv('filesnames/trainset_10_uids.csv')
valset = pd.read_csv('filesnames/valset_10_uids.csv')

trainset = trainset.drop('Unnamed: 0', axis=1)
valset = valset.drop('Unnamed: 0', axis=1)

trainset_dl = [tuple(x) for x in trainset.values]
valset_dl = [tuple(x) for x in valset.values]

train_filepaths = [i[2] for i in trainset_dl]
train_labels = [i[3] for i in trainset_dl]
valid_filepaths = [i[2] for i in valset_dl]
val_labels = [i[3] for i in valset_dl]

In [29]:
# data loaders

## mfcc ##
# mean=-3.121299957927269
# std=50.02533504630946

## spectrograms ##
mean=-3.1259581955996425
std=0.8961027914827521

batch_size=16
num_workers=8
data_train_sub = CustomDatasetSimple(train_filepaths, train_labels, mean, std)
data_valid_sub = CustomDatasetSimple(valid_filepaths, val_labels, mean, std)

train_data_loader = torch.utils.data.DataLoader(
    dataset=data_train_sub, batch_size=batch_size, shuffle=True,
    num_workers=num_workers)

valid_data_loader = torch.utils.data.DataLoader(
    dataset=data_valid_sub, batch_size=batch_size, shuffle=True,
    num_workers=num_workers)

In [33]:
# attack functions

def pgd_linf_targ_own(model, X, y, epsilon=0.1, alpha=0.01, num_iter=20, randomize=False, y_targ=None):
    """ Construct PGD adversarial examples on the examples X"""
    np_y_targ = np.ones_like(y.detach().cpu().numpy())*y_targ
    t_y_targ = torch.from_numpy(np_y_targ).cuda()
    if randomize:
        delta = torch.rand_like(X, requires_grad=True)
        delta.data = delta.data * 2 * epsilon - epsilon
    else:
        delta = torch.zeros_like(X, requires_grad=True)
    print(t_y_targ)
    for t in range(num_iter):
        loss = nn.CrossEntropyLoss()(model(X + delta), y)-nn.CrossEntropyLoss()(model(X + delta), t_y_targ)
        loss.backward()
        delta.data = (delta + alpha*delta.grad.detach().sign()).clamp(-epsilon,epsilon)
        delta.grad.zero_()
    return delta.detach()

def fgsm_targ(model, X, y, epsilon=0.3, alpha=None, num_iter=None):
    """ Construct FGSM adversarial examples on the examples X"""
    delta = torch.zeros_like(X, requires_grad=True)
    delta = delta.cuda()
    loss = nn.CrossEntropyLoss()(model(X + delta), y)
    loss.backward()
    return epsilon * delta.grad.detach().sign()

### Spectrograms -- Target Attacks

In [51]:
# load model

model_name = '10_uids_mfcc_resnet_norm_pgd_0.1_0.02_10_epochs_128'
criterion = torch.nn.CrossEntropyLoss()
model = resnet34(pretrained=False, progress=False).cuda()

spectrogram=False

if spectrogram:

    model = torch.load('saved/' + str(model_name))
    
else:

    model.load_state_dict(torch.load('saved/' + str(model_name)))
    LOGFILE_PATH = model_name + '_eval'

LOGFILE_PATH = model_name + '_eval'
trainer = BaseTrainer(model=model, 
                  train_dl=train_data_loader, 
                  valid_dl=valid_data_loader, 
                  criterion=criterion, 
                  model_filename=model_name, 
                  n_epochs=3)

In [52]:
list_y_tar_pred = {}
list_y_true = {}
for i in range(10):
    list_y_tar_pred[i] = []
    list_y_true[i] = []
    for X, y_true in trainer.valid_dl:
        X = X.cuda()
        y_true = y_true.cuda()
        delta = pgd_linf_targ_own(trainer.model, X, y_true, 0.1, 0.01, 60, y_targ=i)
        delta = delta.cuda()
        y_pred = trainer.model(X + delta)
        y_pred = torch.argmax(y_pred, axis=1)
        list_y_tar_pred[i] += list(y_pred.detach().cpu().numpy())
        list_y_true[i] += list(y_true.detach().cpu().numpy())
    print(i)
    np_tmp1 = np.asarray(list_y_tar_pred[i])
    np_tmp2 = np.asarray(list_y_true[i])
    print(np.mean(np_tmp1[np_tmp2!=i]==i))

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0, 0], device='cuda:0')
0
0.37777777777777777
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,